#### 在Cox PH Model下產生存活資料並求出參數的MLE

1.在 Cox PH Model 下的 hazard function 為：

$$
h(t) = h_0(t)e^{\beta'Z}
$$

又可知：

$$
h(t) = -\frac{d}{dt}log(S(t))
$$

即可求出 survival function 為：

$$
1 - F(t) = S(t) = e^{-\int^t_0 h(x)dx}
$$


$$
1 - F(t) = S(t) = e^{-\int^t_0 h(x)dx} = e^{-\int^t_0 h_0(x)e^{\beta 'Z} dx} = e^{-e^{\beta'Z}\int^t_0 h_0(x)dx}\\
= e^{-e^{\beta'Z}H_0(t)},\,\,where\,\,\small{H_0(t) =  \int^t_0 h_0(x)dx}
$$

$$
F(t) = 1 - e^{-e^{\beta'Z}H_0(t)} \sim U(0, 1) \equiv U
$$

2.模擬步驟：

+ 用 Bernoulli 和 N(0, 1) 分別生成兩個 Covariates Z1, Z2
+ 生成需要資料數目的均勻亂數
+ 基於機率積分轉換，使用目標 CDF 的反函數，並帶入所生成的均勻亂數，即可得到模擬的設限時間
+ 從上面可以知道設限時間的 CDF 與 Cox PH Model 的關係

$$
T = H^{-1}_0(-log(1 - U) \times e^{-\beta'Z})
$$

+ 設限時間的部分則採用指數分配生成。
+ 設定 $\beta_1$, $\beta_2$, $h_0(t)$，這邊設定 $\beta_1 = 0.5$、 $\beta_2 = -0.5$，$h_0(t)$ 的部分考慮兩種情況，一種為固定常數c，另一種為與時間呈現線性關係，即 $h_0(t) = ct$，在產生資料的過程中，$c$ 令為 1。
+ 生成資料後，將資料帶入 likelihood or log-likelihood function
$$
\normalsize{L = \Pi^n_{i = 1}(f(t_i))^{\delta_i} (S(t_i))^{1 - \delta_i}}\\
  \normalsize{= \Pi^n_{i = 1} S(t_i)(h(t_i))^{\delta_i}}\\
  \normalsize{= \Pi^n_{i = 1}e^{-e^{\beta'Z}\int^t_0 h_0(x)dx} (h_0(t_i)e^{\beta'Z})^{\delta_i}}
$$

$$
log L(\beta_1, \beta_2, c) = \sum^n_{i = 1}(-e^{\beta'Z}\int^t_0 h_0(x)dx + \delta_i (\beta'Z + log(h_0(t)))
$$

+ 再使用 `R` 內建的指令，求取出 MLE。
+ p.s. Event Time 的資料量必須超過總資料量的 60%
  
---

#### 以下針對$\beta_1$, $\beta_2$, $h_0(t) = c$ 的部分做模擬並求出MLE：

$$
H_0(t) = \int_0^t c dx = ct\\
F(t) = 1 - e^{-e^{\beta'Z} ct} \sim U(0, 1) \equiv U\\
T = \frac{-1}{c}log(1 - U) e^{-\beta' Z}
$$

In [248]:
# generate data under h_0(t) = c (constant)
generate.data.c = function(N, par){
    # generate covariates z1 and z2 by one N(0, 1) and one Ber(p = 0.5)
    z = cbind(z1 = rnorm(N), z2 = sample(c(0, 1), N, replace = T, prob = c(.5, .5)))
    # generate uniform random number
    u = runif(N)
    # check the form of inverse cdf of event time
    cdf.inverse = function(u, par){
        return (return (-log(1 - u) * exp(-1 * (par[1] * z[1:N, 1] + par[2] * z[1:N, 2])) / par[3]))
    }
    # generate event time
    e = cdf.inverse(u, par)
    # generate censored
    c = rexp(N, rate = 0.5)
    # combine event time and censored
    data = cbind(e, c)
    # find the observation by min(event time, censored)
    obs = c()
    # if the min is event time, the delta is 1
    # if the min is censored, the delta is 0
    delta = c()
    # use for loop to find observation and delta
    for (i in 1:N){
        if (data[i, 1] < data[i, 2]){
            obs = c(obs, data[i, 1])
            delta = c(delta, 1)
        }else{
            obs = c(obs, data[i, 2])
            delta = c(delta, 0)
        }
    }
    # combine all data
    data = cbind(data, obs, delta, z)
    # delete the original event time and censored, we only need the observation time, delta and covariates
    data = data[, -c(1, 2)]
    # rename the column
    colnames(data) = c("Observation", "Delta", "Z1", "Z2")
    # reset the row index
    row.names(data) = NULL
    # return data
    return (data)
}

In [249]:
# generate data 1000 times to check whether the mean of the number of event times over 60% of all data
value = c()
for (i in 1:1000){
    dt = generate.data.c(N = 1000, par = c(0.5, -0.5, 1))
    value = c(value, sum(dt[, 2]))
}
print(mean(value))

[1] 602.414


$$
log L(\beta_1, \beta_2, c) = \sum^n_{i=1}(-ct e^{\beta_1 Z_1 + \beta_2 Z_2} + \delta_i(\beta_1 Z_1 + \beta_2 Z_2 + log(c)))
$$

In [250]:
# do the simulation with finding mle
mle.simulation.c = function(n, N, par, init){
    # create an empty vector to store mle
    mle.vector = c()
    # write down the loglikelihood form
    f = function(pa, data){
        data = data
        l = sum(pa[3] * data[, 1] * exp(pa[1] * data[, 3] + pa[2] * data[, 4]) - data[, 2] * (pa[1] * data[, 3] + pa[2] * data[, 4] + log(pa[3])))
    }
    
    for (i in 1:n){   
        # generate the data
        dt = generate.data.c(N, par)
        # maximize the parameters
        mle = optim(par = init, fn = f, data = dt)
        # put the mle into the above empty vector
        mle.vector = rbind(mle.vector, mle$par)
    }
    
    # rename the column
    colnames(mle.vector) = c("beta 1", "beta 2", "c")
    # return the vector of mle
    return (mle.vector)
}

In [251]:
result.c = mle.simulation.c(1000, 1000, par = c(0.5 ,-0.5, 1), init = c(1, 1, 1))
print(apply(result.c, 2, mean))
print(apply(result.c, 2, var))

    beta 1     beta 2          c 
 0.5007196 -0.5004100  1.0017684 
     beta 1      beta 2           c 
0.001719924 0.007302031 0.003181699 


$$
\frac{\partial}{\partial \beta_1}logL = \sum^n_{i=1}(-ct Z_1 e^{\beta_1 Z_1 + \beta_2 Z_2} + \delta_i Z_1)\\
\frac{\partial}{\partial \beta_2}logL = \sum^n_{i=1}(-ct Z_2 e^{\beta_1 Z_1 + \beta_2 Z_2} + \delta_i Z_2)\\
\frac{\partial}{\partial c}logL = \sum^n_{i=1}(-t e^{\beta_1 Z_1 + \beta_2 Z_2} + \frac{\delta_i}{c})
$$

In [252]:
install.packages(c("BB", "rootSolve"))
library(BB)
library(rootSolve)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [253]:
# using score to calculate the mle
solve.first.order.sim.c = function(n, par, fn, init){
    library(rootSolve)
    # create an empty vector to store the mle
    mle.vec = c()
    
    for (i in 1:n){
        # generate data
        dt = generate.data.c(n, par)
        # there are many problems in multiroot command, sometimes we can't calculate the true value
        # the command will show the warning, we just need the correct values
        # the `tryCatch` command catches the error warning message
        mle = tryCatch(multiroot(fn, init, data = dt), error = identity, warning = identity)
        # if we have warning or error message, we discard the values of this turn, and do the next turn 
        if (is(mle, c("warning"))){
            next
        }else if (is(mle, c("error"))){
            next
        }else{
            mle.vec = rbind(mle.vec, mle$root)
        }
    }
    # there's no warning when the NaN produced, so we omit it
    mle.vec = na.omit(mle.vec)
    # change the data structure
    mle.vec = data.frame(mle.vec)
    # rename the columns
    colnames(mle.vec) = c("beta 1", "beta 2", "c")
    # return the vector
    return (mle.vec)
}

In [254]:
# write down the equations we want to solve
f = function(par, data){
    b1 = par[1]
    b2 = par[2]
    c = par[3]
    c(f1 = sum(-c * data[, 1] * data[, 3] * exp(b1 * data[, 3] + b2 * data[, 4]) + data[, 2] * data[, 3]),
      f2 = sum(-c * data[, 1] * data[, 4] * exp(b1 * data[, 3] + b2 * data[, 4]) + data[, 2] * data[, 4]),
      f3 = sum(-data[, 1] * exp(b1 * data[, 3] + b2 * data[, 4]) + (data[, 2] / c)))
}
# using the above defined function to solve the equations
result = solve.first.order.sim.c(n = 1000, par = c(0.5, -0.5, 1), fn = f, init = c(1, 1, 1))
# calculate the mean and variance of the solved mle
print(apply(result, 2, mean))
print(apply(result, 2, var))

    beta 1     beta 2          c 
 0.4981746 -0.5084700  1.0082453 
     beta 1      beta 2           c 
0.001826226 0.005263657 0.002347510 


$$
\frac{\partial^2}{\partial \beta_1^2}logL = \sum^n_{i=1}(-ctZ_1^2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_1 \partial \beta_2}logL = \sum^n_{i=1}(-ctZ_1Z_2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_1 \partial c}logL = \sum^n_{i=1}(-tZ_1 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_2^2}logL = \sum^n_{i=1}(-ctZ_2^2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_2 \partial c}logL = \sum^n_{i=1}(-tZ_2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial c^2}logL = \sum^n_{i=1}(-\frac{\delta_i}{c^2})\\
$$

In [255]:
# write down the element of information matrix
I11 = function(data, par){
    return (sum(par[3] * data[, 1] * (data[, 3]^2) * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I12 = function(data, par){
    return (sum(par[3] * data[, 1] * data[, 3] * data[, 4] * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I13 = function(data, par){
    return (sum(data[, 1] * data[, 3] * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I22 = function(data, par){
    return (sum(par[3] * data[, 1] * (data[, 4]^2) * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I23 = function(data, par){
    return (sum(data[, 1] * data[, 4] * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I33 = function(data, par){
    return (sum(data[, 2] / par[3]))
}

In [256]:
# simulate infromation matrix many times and find the variance of mle of parameters
sim.var.c = function(n, N, par){
    # create an empty vector
    var.vec = c()
    # in this for loop
    # generate data, and put the data & parameter(manual setting) into the defined funciton
    for (i in 1:n){
        data = generate.data.c(N, par)
        i11 = I11(data, par); i12 = I12(data, par); i13 = I13(data, par)
        i22 = I22(data, par); i23 = I23(data, par); i33 = I33(data, par)
        # let the i11 ~ i33 arrange as a matrix
        mat = matrix(c(i11, i12, i13,
                       i12, i22, i23,
                       i13, i23, i33), ncol = 3, byrow = T)
        # find the inverse
        mat.inv = solve(mat)
        # the diagonal of the inverse matrix is the variance of mle of parameters
        d = diag(mat.inv)
        var.vec = rbind(var.vec, d)
    }
    # reset the row index
    row.names(var.vec) = NULL
    # rename the columns
    colnames(var.vec) = c("beta 1", "beta 2", "c")
    return (var.vec)
}

In [257]:
var.c = sim.var.c(1000, 1000, par = c(0.5, -0.5, 1))
print(apply(var.c, 2, mean))

     beta 1      beta 2           c 
0.001762066 0.006746823 0.003105305 


---

#### 以下針對$\beta_1$, $\beta_2$, $h_0(t) = ct$ 的部分做模擬並求出MLE：

$$
H_0(t) = \int^t_0 cx dx = \frac{ct^2}{2}\\
F(t) = 1 - e^{-exp(\beta'Z) \frac{ct^2}{2}} \sim U(0, 1) \equiv U\\
T = \sqrt{\frac{-2}{c}log(1 - U) e^{-\beta' Z}}
$$

In [258]:
generate.data.ct = function(N, par){
    # generate covariates z1 and z2
    z = cbind(z1 = rnorm(N), z2 = sample(c(0, 1), N, replace = T, prob = c(.5, .5)))
    # generate the uniform random numbers
    u = runif(N)
    # write down the form of cdf inverse
    cdf.inverse = function(u, par){
        return (sqrt(-2 * log(1 - u) * exp(-1 * (par[1] * z[1:N, 1] + par[2] * z[1:N, 2])) / par[3]))
    }
    # using inverse method to generate event time
    e = cdf.inverse(u, par)
    # generate censored
    c = rexp(N, rate = 0.3)
    # combine the event time and censored
    data = cbind(e, c)
    # observation is min(event time, censored)
    obs = c()
    # if the min is event time, then delta is 1
    # if the min is censored, then delta is 0
    delta = c()
    for (i in 1:N){
        if (data[i, 1] < data[i, 2]){
            obs = c(obs, data[i, 1])
            delta = c(delta, 1)
        }else{
            obs = c(obs, data[i, 2])
            delta = c(delta, 0)
        }
    }
    # combine the data
    data = cbind(data, obs, delta, z)
    # we just need the observation time and delta and covariates
    data = data[, -c(1, 2)]
    # rename the column
    colnames(data) = c("Observation", "Delta", "Z1", "Z2")
    # reset the index
    row.names(data) = NULL
    return (data)
}


In [259]:
value = c()
for (i in 1:1000){
    dt = generate.data.ct(N = 1000, par = c(0.5, -0.5, 1))
    value = c(value, sum(dt[, 2]))
}

print(mean(value))

[1] 663.432


In [260]:
print(generate.data.ct(10, par = c(0.5, -0.5, 1)))

      Observation Delta          Z1 Z2
 [1,]  0.30082030     1  0.45858598  0
 [2,]  0.06216375     0  0.39565538  0
 [3,]  1.38016177     1 -0.01756629  0
 [4,]  0.27570969     1  1.63029115  0
 [5,]  1.25501768     1  0.93604235  1
 [6,]  0.51073747     0 -0.56829060  1
 [7,]  1.42161259     0 -1.06579706  1
 [8,]  0.54197148     1 -0.02275361  0
 [9,]  0.13810552     1 -1.08399584  0
[10,]  1.24047401     1  1.39645336  1


$$
log L(\beta_1, \beta_2, c) = \sum^n_{i=1}(-\frac{ct^2}{2}e^{\beta_1 Z_1 + \beta_2 Z_2} + \delta_i(\beta_1 Z_1 + \beta_2 Z_2 + log(ct)))
$$

In [261]:
mle.simulation.ct = function(n, N, par, init){
    # n：要迴圈幾次
    # N：每一次迴圈生成多少樣本
    # par：自行挑選參數，用來生成資料
    # init：給予 optim 的起始值
    
    # 創建空向量，等等把算出來的 mle 放進去排好
    mle.vector = c()
    
    # 要極大化參數的目標函數
    # optim 本身是最小化，所以loglikelihood裡面要乘上一個(-1)
    f = function(pa, data){
        data = data
        l = sum(0.5 * pa[3] * (data[, 1]^2) * exp(pa[1] * data[, 3] + pa[2] * data[, 4]) - data[, 2] * (pa[1] * data[, 3] + pa[2] * data[, 4] + log(pa[3] * data[, 1])))
    }
    
    # 利用迴圈決定要做幾次
    for (i in 1:n){
        # 利用前面定義的函數生出資料
        dt = generate.data.ct(N, par)
        mle = optim(par = init, fn = f, data = dt)
        mle.vector = rbind(mle.vector, mle$par)
    }
    # 重新命名 mle 向量的每一行
    colnames(mle.vector) = c("beta 1", "beta 2", "c")
    return (mle.vector)
}

In [262]:
result.ct = mle.simulation.ct(1000, 1000, par = c(0.5 ,-0.5, 1), init = c(1, 1, 1))
print(apply(result.ct, 2, mean))
print(apply(result.ct, 2, var))

Warning message in log(pa[3] * data[, 1]):
“NaNs produced”
Warning message in log(pa[3] * data[, 1]):
“NaNs produced”
Warning message in log(pa[3] * data[, 1]):
“NaNs produced”
Warning message in log(pa[3] * data[, 1]):
“NaNs produced”
Warning message in log(pa[3] * data[, 1]):
“NaNs produced”


    beta 1     beta 2          c 
 0.5009333 -0.4988024  1.0009079 
     beta 1      beta 2           c 
0.001610224 0.005984680 0.003015429 


$$
\frac{\partial}{\partial \beta_1}logL = \sum^n_{i=1}(-\frac{ct^2}{2} Z_1 e^{\beta_1 Z_1 + \beta_2 Z_2} + \delta_i Z_1)\\
\frac{\partial}{\partial \beta_2}logL = \sum^n_{i=1}(-\frac{ct^2}{2} Z_2 e^{\beta_1 Z_1 + \beta_2 Z_2} + \delta_i Z_2)\\
\frac{\partial}{\partial c}logL = \sum^n_{i=1}(-\frac{t^2}{2} e^{\beta_1 Z_1 + \beta_2 Z_2} + \frac{\delta_i}{c})\\
$$

In [263]:
install.packages(c("BB", "rootSolve"))
library(BB)
library(rootSolve)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [264]:
# solve the score to find mle
solve.first.order.sim.ct = function(n, par, fn, init){
    library(rootSolve)
    
    mle.vec = c()
    for (i in 1:n){
        dt = generate.data.ct(n, par)
        mle = tryCatch(multiroot(fn, init, data = dt), error = identity, warning = identity)
        if (is(mle, c("warning"))){
            next
        }else if (is(mle, c("error"))){
            next
        }else{
            mle.vec = rbind(mle.vec, mle$root)
        }
    }
    
    mle.vec = na.omit(mle.vec)
    mle.vec = data.frame(mle.vec)
    colnames(mle.vec) = c("beta 1", "beta 2", "c")
    return (mle.vec)
}

In [265]:
f = function(par, data){
    b1 = par[1]
    b2 = par[2]
    c = par[3]
    c(f1 = sum(-0.5 * c * (data[, 1]^2) * data[, 3] * exp(b1 * data[, 3] + b2 * data[, 4]) + data[, 2] * data[, 3]),
      f2 = sum(-0.5 * c * (data[, 1]^2) * data[, 4] * exp(b1 * data[, 3] + b2 * data[, 4]) + data[, 2] * data[, 4]),
      f3 = sum(-0.5 * (data[, 1]^2) * exp(b1 * data[, 3] + b2 * data[, 4]) + (data[, 2] / c)))
}

result = solve.first.order.sim.ct(n = 1000, par = c(0.5, -0.5, 1), fn = f, init = c(1, 1, 1))
print(apply(result, 2, mean))
print(apply(result, 2, var))

    beta 1     beta 2          c 
 0.4986830 -0.5091216  1.0079151 
     beta 1      beta 2           c 
0.001642902 0.004932841 0.002287421 


$$
\frac{\partial^2}{\partial \beta_1^2}logL = \sum^n_{i=1}(-\frac{ct^2}{2}Z_1^2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_1 \partial \beta_2}logL = \sum^n_{i=1}(-\frac{ct^2}{2}Z_1Z_2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_1 \partial c}logL = \sum^n_{i=1}(-\frac{t^2}{2} Z_1 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_2^2}logL = \sum^n_{i=1}(-\frac{ct^2}{2}Z_2^2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial \beta_2 \partial c}logL = \sum^n_{i=1}(-\frac{t^2}{2}Z_2 e^{\beta_1 Z_1 + \beta_2 Z_2})\\
\frac{\partial^2}{\partial c^2}logL = \sum^n_{i=1}(-\frac{\delta_i}{c^2})\\
$$

In [266]:
I11 = function(data, par){
    return (sum(0.5 * par[3] * (data[, 1]^2) * (data[, 3]^2) * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I12 = function(data, par){
    return (sum(0.5 * par[3] * (data[, 1]^2) * data[, 3] * data[, 4] * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I13 = function(data, par){
    return (sum(0.5 * (data[, 1]^2) * data[, 3] * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I22 = function(data, par){
    return (sum(0.5 * par[3] * (data[, 1]^2) * (data[, 4]^2) * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I23 = function(data, par){
    return (sum(0.5 * (data[, 1]^2) * data[, 4] * exp(par[1] * data[, 3] + par[2] * data[, 4])))
}

I33 = function(data, par){
    return (sum(data[, 2] / par[3]))
}

In [267]:
sim.var.ct = function(n, N, par){
    var.vec = c()
    for (i in 1:n){
        data = generate.data.ct(N, par)
        i11 = I11(data, par); i12 = I12(data, par); i13 = I13(data, par)
        i22 = I22(data, par); i23 = I23(data, par); i33 = I33(data, par)
        mat = matrix(c(i11, i12, i13,
                       i12, i22, i23,
                       i13, i23, i33), ncol = 3, byrow = T)
        mat.inv = solve(mat)
        d = diag(mat.inv)
        var.vec = rbind(var.vec, d)
    }
    row.names(var.vec) = NULL
    colnames(var.vec) = c("beta 1", "beta 2", "c")
    return (var.vec)
}

In [268]:
var.ct = sim.var.ct(1000, 1000, par = c(0.5, -0.5, 1))
print(apply(var.ct, 2, mean))

     beta 1      beta 2           c 
0.001560268 0.006079000 0.002911913 
